# Guided Project: Predicting House Sale Prices

In this course, we started by building intuition for model based learning, explored how the linear regression model worked, understood how the two different approaches to model fitting worked, and some techniques for cleaning, transforming, and selecting features. In this guided project, you can practice what you learned in this course by exploring ways to improve the models we built.

You'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can read more about why the data was collected [here](https://doi.org/10.1080/10691898.2011.11889627). You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

Let's start by setting up a pipeline of functions that will let us quickly iterate on different models
- train
- transform_features()
- select_features()
- train_and_test()

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

In [2]:
# Read file into dataframe
pd.options.display.max_columns = 99
housing = pd.read_csv('AmesHousing.tsv', sep='\t')
housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
housing.shape

(2930, 82)

In [4]:
def transform_features(df):
    # 70% of 2930
#     train_set = df[:2051]
    return df

In [5]:
def select_features(df):
    # Gr Liv Area and SalePrice columns
    return df[['Gr Liv Area', 'SalePrice']]

In [6]:
def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['int64', 'float64'])
    numeric_test = test.select_dtypes(include=['int64', 'float64'])
    
    target = 'SalePrice'
    features = numeric_train.columns.drop(target)
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    prediction = lr.predict(test[features])
    mse = mean_squared_error(test[target], prediction)
    rmse = mse ** (1/2)
    return rmse

In [7]:
train = transform_features(housing)
selected_features = select_features(train)
first_rmse = train_and_test(selected_features)
first_rmse

57088.25161263909

Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns. Update transform_features() so that any column from the data frame with more than 25% (or another cutoff value) missing values is dropped. You also need to remove any columns that leak information about the sale (e.g. like the year the sale happened). In general, the goal of this function is to:

- remove features that we don't want to use in the model, just based on the number of missing values or data leakage
- transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc)
- create new features by combining other features

Next, you need to get more familiar with the remaining columns by reading the [data documentation](https://s3.amazonaws.com/dq-content/307/data_description.txt) for each column, determining what transformations are necessary (if any), and more. As we mentioned earlier, succeeding in predictive modeling (and competitions like Kaggle) is highly dependent on the quality of features the model has. Libraries like scikit-learn have made it quick and easy to simply try and tweak many different models, but cleaning, selecting, and transforming features are still more of an art that requires a bit of human ingenuity.

In [8]:
# Setting the SalePrice df
df_salePrice = housing['SalePrice']

In [9]:
# All numeric columns contain less than 5% missing values
def transform_features_five_percent(df):
    df = housing
    # Getting the percentage of missing values
    df_null_counts = df.isnull().sum() * 100 / len(df)
    # Which columns contains less than 5% missing values
    five_percent_mv = df[df_null_counts[(df_null_counts>0) & (df_null_counts<5)].index]
    # Select numerical columns and fill in the missing values with mean
    numeric_five_percent = five_percent_mv.select_dtypes(include=['int64','float64'])
    numeric_five_percent = numeric_five_percent.fillna(numeric_five_percent.mean())
    return numeric_five_percent

numeric_five_percent = transform_features_five_percent(housing)
# Adding the SalePrice column to less than 5%
five_percent_features = pd.concat([numeric_five_percent, df_salePrice], axis=1)
five_percent_rmse = train_and_test(five_percent_features)
five_percent_rmse

55161.311274313935

In [10]:
# All Numeric columns with non null missing values
def transform_features_all(df):
    # Get summary of null counts
    df_null_counts = df.isnull().sum()
    df_no_mv = df[df_null_counts[df_null_counts==0].index]
    return df_no_mv

all_no_mv = transform_features_all(housing)
all_no_mv_rmse = train_and_test(all_no_mv)
all_no_mv_rmse

80253.44846043282

In [11]:
df = housing
df_null_counts = df.isnull().sum()
df[df_null_counts[df_null_counts==0].index].head()

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Exter Qual,Exter Cond,Foundation,Heating,Heating QC,Central Air,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,TA,TA,CBlock,GasA,Fa,Y,1656,0,0,1656,1,0,3,1,TA,7,Typ,2,P,210,62,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,TA,TA,CBlock,GasA,TA,Y,896,0,0,896,1,0,2,1,TA,5,Typ,0,Y,140,0,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,TA,TA,CBlock,GasA,TA,Y,1329,0,0,1329,1,1,3,1,Gd,6,Typ,0,Y,393,36,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,Gd,TA,CBlock,GasA,Ex,Y,2110,0,0,2110,2,1,3,1,Ex,8,Typ,2,Y,0,0,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,TA,TA,PConc,GasA,Gd,Y,928,701,0,1629,2,1,3,1,TA,6,Typ,1,Y,212,34,0,0,0,0,0,3,2010,WD,Normal,189900


In [12]:
df[df_null_counts[df_null_counts!=0].index].head()

,Lot Frontage,Alley,Mas Vnr Type,Mas Vnr Area,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Electrical,Bsmt Full Bath,Bsmt Half Bath,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Pool QC,Fence,Misc Feature
0,141.0,NaN,Stone,112.0,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,SBrkr,1.0,0.0,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,NaN,NaN,NaN
1,80.0,NaN,None,0.0,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,SBrkr,0.0,0.0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,NaN,MnPrv,NaN
2,81.0,NaN,BrkFace,108.0,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,SBrkr,0.0,0.0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,NaN,NaN,Gar2
3,93.0,NaN,None,0.0,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,SBrkr,1.0,0.0,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,NaN,NaN,NaN
4,74.0,NaN,None,0.0,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,SBrkr,0.0,0.0,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,NaN,MnPrv,NaN


In [13]:
# Columns I think that should be features for the model
custom = pd.DataFrame()
custom['years_until_remod'] = housing['Year Remod/Add'] - housing['Year Built']
import_features = housing[['Lot Area', 'Overall Qual', 'Overall Cond', 
                   'Gr Liv Area', 'Total Bsmt SF', 'Garage Area','SalePrice']]
custom_features = pd.concat([import_features, custom], axis=1)
# Dropping the missing values
custom_features = custom_features.dropna(axis=0)
custom_rmse = train_and_test(custom_features)
custom_rmse

41558.55287375407

In [14]:
# Normalize the Custom features
price_col = custom_features['SalePrice']
normalize_features = (custom_features-custom_features.min())/(
    custom_features.max()-custom_features.min())
normalize_features['SalePrice'] = price_col
normalize_rmse = train_and_test(normalize_features)
normalize_rmse

41558.55287375385

The results are the same without normalizing the columns

In [15]:
corrmat = custom_features.corr()
sorted_corrmat = corrmat['SalePrice'].abs().sort_values()
print(sorted_corrmat)

Overall Cond         0.101146
years_until_remod    0.240354
Lot Area             0.266393
Total Bsmt SF        0.632260
Garage Area          0.640276
Gr Liv Area          0.706779
Overall Qual         0.799140
SalePrice            1.000000
Name: SalePrice, dtype: float64


In [16]:
# Top 4 correlation with SalePrice
final_corr_cols = sorted_corrmat.drop(['Overall Cond', 'years_until_remod', 'Lot Area'])
final_corr_features = custom_features[final_corr_cols.index]
final_corr_rmse = train_and_test(final_corr_features)
final_corr_rmse

41919.146069909635

In [17]:
# All numeric columns correlate to SalePrice
all_numeric_corr = df[df_null_counts[df_null_counts==0].index]
all_numeric_corr = all_numeric_corr.select_dtypes(include=['int64', 'float64'])
all_corrmat = all_numeric_corr.corr()
all_sorted_corrmat = all_corrmat['SalePrice'].abs().sort_values(ascending=False)
print(all_sorted_corrmat[:5])

SalePrice       1.000000
Overall Qual    0.799262
Gr Liv Area     0.706780
1st Flr SF      0.621676
Year Built      0.558426
Name: SalePrice, dtype: float64


Based on the two testing of correlation, the top two columns correlated to SalepPrice are Overall Qual and Gr Liv Area

In [18]:
# Getting and setting dummy variables
dummy_df = pd.DataFrame()
dummy_df = pd.get_dummies(df['Overall Qual'])
dummy_df = pd.concat([dummy_df, pd.get_dummies(df['Gr Liv Area'])], axis=1)
dummy_df = pd.concat([dummy_df, df_salePrice], axis=1)
dummy_df = dummy_df.astype('float')
dummy_rmse = train_and_test(dummy_df)
dummy_rmse

7.043518331165266e+17

In [19]:
def train_and_test(df, k=0):
    if k == 1:
        fold_one = df[:1460]
        fold_two = df[1460:]
        split_one = fold_one
        test_one = fold_two
        split_two = fold_two
        test_two = fold_one
        
        target = 'SalePrice'
        features = fold_one.columns.drop(target)
        
        lr = LinearRegression()
        lr.fit(fold_one[features], fold_one[target])
        prediction1 = lr.predict(fold_two[features])
        mse1 = mean_squared_error(fold_two[target], prediction1)
        rmse1 = mse1 ** (1/2)
        
        lr2 = LinearRegression()
        lr2.fit(fold_two[features], fold_two[target])
        prediction2 = lr2.predict(fold_one[features])
        mse2 = mean_squared_error(fold_one[target], prediction2)
        rmse2 = mse2 ** (1/2)
        
        avg_mse = np.mean([rmse1, rmse2])
        return avg_mse
    else:
        train = df[:1460]
        test = df[1460:]
        
        target = 'SalePrice'
        features = train.columns.drop(target)
        
        kf = KFold(k, shuffle=True, random_state=1)
        lr = LinearRegression()
        mses = cross_val_score(lr, train[features], train[target], scoring="neg_mean_squared_error", cv=kf)
        rmses = np.sqrt(np.absolute(mses))
        avg_rmse = np.mean(rmses)
        return avg_rmse       

### Selected features Gr Liv Area and SalePrice columns

In [20]:
train_and_test(selected_features, 1)

56642.756239856666

In [21]:
train_and_test(selected_features, 10)

55741.41087845804

### Columns that are less than 5 percent missing values

In [22]:
train_and_test(five_percent_features, 1)

50649.53178932025

In [23]:
train_and_test(five_percent_features, 10)

44749.81394293866

### Columns I think that should be features for the model

In [24]:
train_and_test(custom_features, 1)

38157.50285328127

In [25]:
train_and_test(custom_features, 10)

33271.492949703665

### Top 4 correlation with SalePrice

In [26]:
train_and_test(final_corr_features, 1)

38722.14647235338

In [27]:
train_and_test(final_corr_features, 10)

33672.17313069407

### Top 2 Correlated columns

In [28]:
train_and_test(df[['Gr Liv Area', 'Overall Qual', 'SalePrice']], 1)

41403.218677367215

In [29]:
train_and_test(df[['Gr Liv Area', 'Overall Qual', 'SalePrice']], 10)

40189.63600729116